# Partycjonowanie danych w Sparku

Partycjonowanie danych jest kluczową techniką zarządzania i optymalizacji przetwarzania dużych zbiorów danych, szczególnie w środowiskach Big Data, takich jak Apache Spark. 
Polega ono na podziale dużego zbioru danych na mniejsze, zarządzalne części, zwane partycjami. 
Każda partycja może być przetwarzana niezależnie od innych, co umożliwia równoległe i rozproszone przetwarzanie na wielu węzłach klastra.

## Cele partycjonowania:
1. Zwiększenie wydajności: Przetwarzając dane w mniejszych blokach równolegle, możemy znacząco przyspieszyć czas przetwarzania zadań.
2. Optymalizacja zapytań: Partycjonowanie pozwala na fizyczne grupowanie danych w sposób, który jest zgodny z najczęściej wykonywanymi zapytaniami. Na przykład, partycjonowanie danych według daty pozwala na szybkie wykonywanie zapytań dotyczących określonych okresów czasu, bez konieczności przeszukiwania całego zbioru danych.
3. Efektywność zarządzania danymi: Partycjonowanie ułatwia zarządzanie danymi, umożliwiając na przykład łatwiejsze archiwizowanie starych danych czy usunięcie niepotrzebnych partycji bez wpływu na resztę zbioru danych.

## Typy partycjonowania:
1. Partycjonowanie poziome (Partitioning): Dane są dzielone w oparciu o wartość jednego lub więcej kolumn. Na przykład, dane mogą być partycjonowane po kolumnie daty lub regionu. 
   W Apache Spark każda partycja jest przypisana do różnych węzłów w klastrze, co umożliwia równoległe przetwarzanie.
2. Partycjonowanie pionowe (Sharding): Polega na podziale kolumn tabeli na różne serwery. Każdy serwer przechowuje tylko określoną grupę kolumn, co może być przydatne w bazach danych, gdzie różne kolumny są często używane przez różne zapytania.



In [3]:
from pyspark.sql import SparkSession

In [4]:
# obiekt sesji zwykle ma nazwę "spark"
spark = SparkSession.builder.appName("SparkPartitioning").getOrCreate() 
spark

In [16]:
from pyspark.sql import DataFrame
from os import listdir

def list_parquet_files(folder: str) -> list[str]:
    return [file for file in listdir(folder) if file.endswith(".parquet")]

def read_parquet_files(folder: str) -> list[DataFrame]:
    files = list_parquet_files(folder)
    return [spark.read.parquet(f"{folder}/{file}") for file in files]

def read_parquet_folder(parquet_files_folder: str) -> DataFrame:
    dfs = read_parquet_files(parquet_files_folder)
    df = dfs[0]
    for i in range(1, len(dfs)):
        df = df.union(dfs[i])
    return df


137766

In [17]:
orders = spark.read.parquet("../../data/sklep/orders")
orders

DataFrame[order_id: int, order_date: bigint, order_customer_id: int, order_status: string]

In [29]:
orders.show(5)

+--------+-------------+-----------------+---------------+
|order_id|   order_date|order_customer_id|   order_status|
+--------+-------------+-----------------+---------------+
|       1|1374735600000|            11599|         CLOSED|
|       2|1374735600000|              256|PENDING_PAYMENT|
|       3|1374735600000|            12111|       COMPLETE|
|       4|1374735600000|             8827|         CLOSED|
|       5|1374735600000|            11318|       COMPLETE|
+--------+-------------+-----------------+---------------+


In [20]:
df = orders.select("order_id", "order_date")
df.show(5)

+--------+-------------+
|order_id|   order_date|
+--------+-------------+
|       1|1374735600000|
|       2|1374735600000|
|       3|1374735600000|
|       4|1374735600000|
|       5|1374735600000|
+--------+-------------+


In [21]:
import pyspark.sql.functions as F

In [22]:
df = df.withColumn("timestamp", F.from_unixtime(F.col("order_date") / 1000))
df.show(5)

+--------+-------------+-------------------+
|order_id|   order_date|          timestamp|
+--------+-------------+-------------------+
|       1|1374735600000|2013-07-25 09:00:00|
|       2|1374735600000|2013-07-25 09:00:00|
|       3|1374735600000|2013-07-25 09:00:00|
|       4|1374735600000|2013-07-25 09:00:00|
|       5|1374735600000|2013-07-25 09:00:00|
+--------+-------------+-------------------+


In [23]:
df = df.withColumn("date_string", F.date_format("timestamp", "yyyy-MM-dd"))
df.show(5)

+--------+-------------+-------------------+-----------+
|order_id|   order_date|          timestamp|date_string|
+--------+-------------+-------------------+-----------+
|       1|1374735600000|2013-07-25 09:00:00| 2013-07-25|
|       2|1374735600000|2013-07-25 09:00:00| 2013-07-25|
|       3|1374735600000|2013-07-25 09:00:00| 2013-07-25|
|       4|1374735600000|2013-07-25 09:00:00| 2013-07-25|
|       5|1374735600000|2013-07-25 09:00:00| 2013-07-25|
+--------+-------------+-------------------+-----------+


In [24]:
df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- date_string: string (nullable = true)


In [26]:
df.write.partitionBy("date_string").mode("overwrite").parquet("orders_partitioned")

In [27]:
spark.read.parquet("orders_partitioned").show(5)

+--------+-------------+-------------------+-----------+
|order_id|   order_date|          timestamp|date_string|
+--------+-------------+-------------------+-----------+
|   15793|1383462000000|2013-11-03 08:00:00| 2013-11-03|
|   15794|1383462000000|2013-11-03 08:00:00| 2013-11-03|
|   15795|1383462000000|2013-11-03 08:00:00| 2013-11-03|
|   15796|1383462000000|2013-11-03 08:00:00| 2013-11-03|
|   15797|1383462000000|2013-11-03 08:00:00| 2013-11-03|
+--------+-------------+-------------------+-----------+


### Partycjonowanie po parze year-month

In [28]:
df = df.withColumn("year_month", F.concat(F.year("timestamp"), F.lit("-"), F.format_string("%02d", F.month("timestamp"))))
df.show(5)

+--------+-------------+-------------------+-----------+----------+
|order_id|   order_date|          timestamp|date_string|year_month|
+--------+-------------+-------------------+-----------+----------+
|       1|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|
|       2|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|
|       3|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|
|       4|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|
|       5|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|
+--------+-------------+-------------------+-----------+----------+


In [30]:
df.write.partitionBy("year_month").mode("overwrite").parquet("orders_partitioned_ym")

### Partycjonowanie po dwóch kolumnach

In [31]:
df = df.withColumn("year", F.year("timestamp"))
df = df.withColumn("month", F.month("timestamp"))

In [32]:
df.show(5)

+--------+-------------+-------------------+-----------+----------+----+-----+
|order_id|   order_date|          timestamp|date_string|year_month|year|month|
+--------+-------------+-------------------+-----------+----------+----+-----+
|       1|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|2013|    7|
|       2|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|2013|    7|
|       3|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|2013|    7|
|       4|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|2013|    7|
|       5|1374735600000|2013-07-25 09:00:00| 2013-07-25|   2013-07|2013|    7|
+--------+-------------+-------------------+-----------+----------+----+-----+


In [33]:
df.write.partitionBy("year", "month").mode("overwrite").parquet("orders_partitioned_ym2")

In [34]:
spark.read.parquet("orders_partitioned_ym2").show(5)

+--------+-------------+-------------------+-----------+----------+----+-----+
|order_id|   order_date|          timestamp|date_string|year_month|year|month|
+--------+-------------+-------------------+-----------+----------+----+-----+
|   15488|1383289200000|2013-11-01 08:00:00| 2013-11-01|   2013-11|2013|   11|
|   15489|1383289200000|2013-11-01 08:00:00| 2013-11-01|   2013-11|2013|   11|
|   15490|1383289200000|2013-11-01 08:00:00| 2013-11-01|   2013-11|2013|   11|
|   15491|1383289200000|2013-11-01 08:00:00| 2013-11-01|   2013-11|2013|   11|
|   15492|1383289200000|2013-11-01 08:00:00| 2013-11-01|   2013-11|2013|   11|
+--------+-------------+-------------------+-----------+----------+----+-----+


In [35]:
df.drop("timestamp").show(5)

+--------+-------------+-----------+----------+----+-----+
|order_id|   order_date|date_string|year_month|year|month|
+--------+-------------+-----------+----------+----+-----+
|       1|1374735600000| 2013-07-25|   2013-07|2013|    7|
|       2|1374735600000| 2013-07-25|   2013-07|2013|    7|
|       3|1374735600000| 2013-07-25|   2013-07|2013|    7|
|       4|1374735600000| 2013-07-25|   2013-07|2013|    7|
|       5|1374735600000| 2013-07-25|   2013-07|2013|    7|
+--------+-------------+-----------+----------+----+-----+
